In [ ]:
# default_exp client

# Client

> client for global prime pay


this client is able to assist you through the GBP payment flow using just python
feel free to comment and contribute

more info at 'https://doc.gbprimepay.com'

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
from globalPrimePay.helper import Keys
from nicHelper.wrappers import add_method
from numbers import Number
import requests

## Client

In [ ]:
#export

class Client:
  '''
    main client for interface with global prime pay
  '''
  def __init__(self, pub, secret, endpoint = 'https://api.globalprimepay.com'):
    '''
      input:
        pub: str:: publiceKey from web
        secret: str:: secretKey from web
      docs (https://doc.gbprimepay.com)
    '''
    self.key = Keys(pub,secret)
    self.endpoint = endpoint
  

## Get token

In [ ]:
#export
@add_method(Client)
def getCardToken(self, cardNumber, expirationMonth, expirationYear, securityCode, name, **kwargs):
  
  body = {
    "rememberCard": True,
    "card": {
      "number": cardNumber,
      "expirationMonth": expirationMonth,
      "expirationYear": expirationYear,
      "securityCode": securityCode,
      "name": name
    }
    ,**kwargs
  }

  headersCharge = {
    'Authorization': self.key.pubPL,
    'Content-Type': 'application/json',
  }
  baseURL = self.endpoint
  url = f'{baseURL}/v2/tokens'
  r = requests.post(url,headers = headersCharge, json = body)
  return r.json()


In [ ]:
#hide

sampleKeys = {
  'pub': "vRvqeoxI1js5CJmURGkiueKtAeGD5rte",
  'secret':  "VWkBurzlZi3dJB78N7jLlztntgDpJSQL"
}
sampleCard = {
      "cardNumber": "4535017710535741",
      "expirationMonth": "05",
      "expirationYear": "28",
      "securityCode": "184",
      "name": "Watcharagon Phokonwong"
    }

In [ ]:
client = Client(**sampleKeys)
r = client.getCardToken(**sampleCard)
print(r)
token = r['card']['token']

{'rememberCard': True, 'resultCode': '00', 'resultMessage': 'Success', 'card': {'name': None, 'number': '453501XXXXXX5741', 'expirationMonth': '05', 'expirationYear': '28', 'securityCode': None, 'token': '9513f0e0-aa69-49bc-98b0-afbd82dab52a', 'cardLocation': 'D', 'cardType': 'VIS'}}


## Charge Card

In [ ]:
@add_method(Client)
def chargeCard(self, token:str, 
               amount:Number, 
               referenceNo:str, 
               customerName:str, 
               customerEmail:str, 
               detail:str,
               responseUrl:str = "https://responseUrl",
               backgroundUrl:str =  "https://backgroundUrl",
               otp:str = "Y",
               **kwargs)->dict:
  body = {
    "amount": amount,
    "referenceNo": referenceNo,
    "detail": detail,
    "customerName": customerName,
    "customerEmail": customerEmail,
    "card": {
      "token": token
    },
    "otp": otp,
    "responseUrl": responseUrl,
    "backgroundUrl": backgroundUrl,
    **kwargs
  }
  
  url = f'{self.endpoint}/v2/tokens/charge'
  headers = {
    'Authorization': self.key.secPL,
    'Content-Type': 'application/json',
  }
  r = requests.post(url, json=body, headers = headers)
  return r.json()


In [ ]:
client = Client(**sampleKeys)
from random import randint

cardPayload = {
  'token': token,
  'amount': 10,
  'referenceNo': f"TEST123{randint(1000,10000)}",
  'customerName': 'nic',
  'customerEmail': 'email@gmail.com',
  'otp': 'Y',
  'detail': 'this is just a test'
}
r = client.chargeCard(**cardPayload)
print(r)
gbpReferenceNo = r['gbpReferenceNo']

{'customerAddress': None, 'amount': 10, 'referenceNo': 'TEST1234975', 'messageToMerchant': None, 'resultCode': '00', 'customerTelephone': None, 'resultMessage': 'Success', 'customerName': 'nic', 'customerEmail': 'email@gmail.com', 'gbpReferenceNo': 'gbp173512057819', 'merchantDefined5': None, 'detail': 'this is just a test', 'merchantDefined3': None, 'merchantDefined4': None, 'merchantDefined1': None, 'merchantDefined2': None}


## Verify OTP

In [ ]:
@add_method(Client)
def verifyOtp(self, gbpReferenceNo):
  params = {
      'publicKey': self.key.pub,
      'gbpReferenceNo': gbpReferenceNo
  }
  headers3d = {
    'Content-Type': 'application/x-www-form-urlencoded',
  }
  url = self.endpoint + "/v2/tokens/3d_secured"
  r = requests.post(url, params=params, headers = headers3d)
  return r.text

In [ ]:
r = client.verifyOtp(gbpReferenceNo)
r

'<!DOCTYPE html>\n\n<html>\n  <head>\n    <title>GB Prime Pay</title>\n    <meta charset="UTF-8" />\n    <meta name="viewport" content="width=device-width, initial-scale=1.0" />\n  </head>\n  <body>\n    <div>Loading...</div>\n    <div>\n      <form action="https://simbank.globalprimepay.com/pay/3d_secure" method="post">\n        <input type="hidden" name="MERID" value="21291800001" />\n        <input type="hidden" name="TERMINALID" value="18060001" />\n        <input type="hidden" name="PAN" value="4535017710535741" />\n        <input type="hidden" name="EXPIRYDATE" value="0528" />\n        <input type="hidden" name="CVV2" value="184" />\n        <input type="hidden" name="INVOICENO" value="2205020023" />\n        <input type="hidden" name="AMOUNT" value="1000" />\n        <input type="hidden" name="POSTURL" value="https://api.globalprimepay.com/web/thanachat_gateway/receive/goback" />\n        <input type="hidden" name="POSTURL2" value="https://api.globalprimepay.com/web/thanachat_ga

In [ ]:
# from IPython.display import HTML
# HTML(r)